# ENGLAND PREMIER LEAGUE SCORE PREDICTION AND EDA
![..](Peoples-Gazette.jpeg)

In [1]:
#IMPORTING LIBRARIES:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import statsmodels.api as sm
from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from warnings import filterwarnings 
filterwarnings('ignore')

# Meanings of columns of dataset:
![DATA](EPL1.png)

In [2]:
#IMPORTING DATASET:

dt=pd.read_csv('EPL_20_21.csv')
df=dt.copy()
df.head()

,Name,Club,Nationality,Position,Age,Matches,Starts,Mins,Goals,Assists,Passes_Attempted,Perc_Passes_Completed,Penalty_Goals,Penalty_Attempted,xG,xA,Yellow_Cards,Red_Cards
0,Mason Mount,Chelsea,ENG,"MF,FW",21,36,32,2890,6,5,1881,82.3,1,1,0.21,0.24,2,0
1,Edouard Mendy,Chelsea,SEN,GK,28,31,31,2745,0,0,1007,84.6,0,0,0.00,0.00,2,0
2,Timo Werner,Chelsea,GER,FW,24,35,29,2602,6,8,826,77.2,0,0,0.41,0.21,2,0
3,Ben Chilwell,Chelsea,ENG,DF,23,27,27,2286,3,5,1806,78.6,0,0,0.10,0.11,3,0
4,Reece James,Chelsea,ENG,DF,20,32,25,2373,1,2,1987,85.0,0,0,0.06,0.12,3,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 532 entries, 0 to 531
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Name                   532 non-null    object 
 1   Club                   532 non-null    object 
 2   Nationality            532 non-null    object 
 3   Position               532 non-null    object 
 4   Age                    532 non-null    int64  
 5   Matches                532 non-null    int64  
 6   Starts                 532 non-null    int64  
 7   Mins                   532 non-null    int64  
 8   Goals                  532 non-null    int64  
 9   Assists                532 non-null    int64  
 10  Passes_Attempted       532 non-null    int64  
 11  Perc_Passes_Completed  532 non-null    float64
 12  Penalty_Goals          532 non-null    int64  
 13  Penalty_Attempted      532 non-null    int64  
 14  xG                     532 non-null    float64
 15  xA    

In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,532.0,25.500000,4.319404,16.0,22.00,26.00,29.000,38.00
Matches,532.0,19.535714,11.840459,1.0,9.00,21.00,30.000,38.00
Starts,532.0,15.714286,11.921161,0.0,4.00,15.00,27.000,38.00
Mins,532.0,1411.443609,1043.171856,1.0,426.00,1345.00,2303.500,3420.00
Goals,532.0,1.853383,3.338009,0.0,0.00,1.00,2.000,23.00
Assists,532.0,1.287594,2.095191,0.0,0.00,0.00,2.000,14.00
Passes_Attempted,532.0,717.750000,631.372522,0.0,171.50,573.50,1129.500,3214.00
Perc_Passes_Completed,532.0,77.823872,13.011631,-1.0,73.50,79.20,84.625,100.00
Penalty_Goals,532.0,0.191729,0.850881,0.0,0.00,0.00,0.000,9.00
Penalty_Attempted,532.0,0.234962,0.975818,0.0,0.00,0.00,0.000,10.00


In [5]:
#Data Wrangling:
df.rename({"xG":"Expected Goals","xA":"Expected Asists","Perc_Passes_Completed":"Pass Accuracy"},axis=1,inplace=True)
df.head()

,Name,Club,Nationality,Position,Age,Matches,Starts,Mins,Goals,Assists,Passes_Attempted,Pass Accuracy,Penalty_Goals,Penalty_Attempted,Expected Goals,Expected Asists,Yellow_Cards,Red_Cards
0,Mason Mount,Chelsea,ENG,"MF,FW",21,36,32,2890,6,5,1881,82.3,1,1,0.21,0.24,2,0
1,Edouard Mendy,Chelsea,SEN,GK,28,31,31,2745,0,0,1007,84.6,0,0,0.00,0.00,2,0
2,Timo Werner,Chelsea,GER,FW,24,35,29,2602,6,8,826,77.2,0,0,0.41,0.21,2,0
3,Ben Chilwell,Chelsea,ENG,DF,23,27,27,2286,3,5,1806,78.6,0,0,0.10,0.11,3,0
4,Reece James,Chelsea,ENG,DF,20,32,25,2373,1,2,1987,85.0,0,0,0.06,0.12,3,0


In [6]:
#Checking for null values:

df.isnull().sum() #No null values

Name                 0
Club                 0
Nationality          0
Position             0
Age                  0
Matches              0
Starts               0
Mins                 0
Goals                0
Assists              0
Passes_Attempted     0
Pass Accuracy        0
Penalty_Goals        0
Penalty_Attempted    0
Expected Goals       0
Expected Asists      0
Yellow_Cards         0
Red_Cards            0
dtype: int64

In [7]:
#Checking for duplicate values:
df.duplicated().sum() #No duplicate values

0

In [8]:
#Creating new columns:

df["MinPerMacth"]=df["Mins"]/df["Matches"] #Minutes per match
df["MinPerGoal"]=df["Mins"]/df["Goals"] #Goals per minute
df.tail()

,Name,Club,Nationality,Position,Age,Matches,Starts,Mins,Goals,Assists,Passes_Attempted,Pass Accuracy,Penalty_Goals,Penalty_Attempted,Expected Goals,Expected Asists,Yellow_Cards,Red_Cards,MinPerMacth,MinPerGoal
527,Lys Mousset,Sheffield United,FRA,"FW,MF",24,11,2,296,0,0,50,80.0,0,0,0.22,0.1,0,0,26.909091,inf
528,Jack O'Connell,Sheffield United,ENG,DF,26,2,2,180,0,0,77,77.9,0,0,0.00,0.0,0,0,90.000000,inf
529,Iliman Ndiaye,Sheffield United,FRA,MF,21,1,0,12,0,0,3,100.0,0,0,0.00,0.0,0,0,12.000000,inf
530,Antwoine Hackford,Sheffield United,ENG,"DF,FW",16,1,0,11,0,0,1,100.0,0,0,1.16,0.0,0,0,11.000000,inf
531,Femi Seriki,Sheffield United,ENG,DF,17,1,0,1,0,0,0,-1.0,0,0,0.00,0.0,0,0,1.000000,inf


In [9]:
#colums of dataset:
df.columns

Index(['Name', 'Club', 'Nationality', 'Position', 'Age', 'Matches', 'Starts',
       'Mins', 'Goals', 'Assists', 'Passes_Attempted', 'Pass Accuracy',
       'Penalty_Goals', 'Penalty_Attempted', 'Expected Goals',
       'Expected Asists', 'Yellow_Cards', 'Red_Cards', 'MinPerMacth',
       'MinPerGoal'],
      dtype='object')

In [10]:
#Exploratory Data Analysis:

#1. Top 10 players with most goals for the season:


top10=df.sort_values(by="Goals",ascending=False).head(10)

px.bar(top10,x="Name",y="Goals",color="Name",title="Top 10 players with most goals for 20-21 Season",
       template="plotly_dark",labels={"Goals":"Goals Scored","Name":"Player Name"})

In [11]:
#2. Top 10 players with most assists for the season:

top10A=df.sort_values(by="Assists",ascending=False).head(20)

px.bar(top10A,x="Name",y="Assists",color="Name",title="Top 10 players with most assists for 20-21 Season",
      labels={"Assists":"Assists","Name":"Player Name"},template="plotly_dark",
      color_discrete_sequence=px.colors.qualitative.Pastel,category_orders={"Position":["FW","MF","DF","GK"]})

In [12]:
#3. Top 10 players with most expected goals for the season:
top10E=df.sort_values(by="Expected Goals",ascending=False).head(10)
px.bar(top10E,x="Name",y="Expected Goals",color="Name",title="Top 10 players with most expected goals for 20-21 Season",labels={"Expected Goals":"Expected Goals","Name":"Player Name"},
       template="plotly_dark") 

In [13]:
#4. Top 10 players with most expected assists for the season:
top10EA=df.sort_values(by="Expected Asists",ascending=False).head(10)
px.bar(top10EA,x="Name",y="Expected Asists",color="Name",title="Top 10 players with most expected assists for 20-21 Season",
       labels={"Expected Asists":"Expected Asists","Name":"Player Name"},template="plotly_dark")

In [14]:
#5. Top 10 players with most passes for the season:

top10P=df.sort_values(by="Passes_Attempted",ascending=False).head(10)

px.bar(top10P,x="Name",y="Passes_Attempted",color="Name",title="Top 10 players with most passes for 20-21 Season",
       labels={"Passes_Attempted":"Passes Attempted","Name":"Player Name"},template="plotly_dark")

In [15]:
#6. correlation between different features:

df_numeric=df.select_dtypes(include=np.number)
fig=px.imshow(df.corr(),title="Correlation between different features",template="plotly_dark",color_continuous_scale=px.colors.sequential.Plasma,width=1000,height=800,
          labels={"index":"Features","value":"Correlation"})
fig.update_layout(xaxis_showgrid=True,yaxis_showgrid=True)


In [16]:
#7. which country has most players in EPL:

nation=df.groupby("Nationality")["Name"].count().reset_index().sort_values(by="Name",ascending=False).head(10)

px.bar(nation,x="Nationality",y="Name",color="Nationality",title="Top 10 countries with most players in EPL",template="plotly_dark",
       labels={"Name":"Count of Players"})



In [17]:
#8. penalty success and failure rate:

Total_penalties=df["Penalty_Attempted"].sum()
failed_penalties=Total_penalties-df["Penalty_Goals"].sum()

px.pie(names=["Penalty Success","Penalty Failure"],values=[df["Penalty_Goals"].sum(),failed_penalties],title="Penalty Success and Failure Rate",
       template="plotly_dark",color_discrete_sequence=px.colors.qualitative.Pastel,width=500,height=500)



In [18]:
#9. clubs with most players in EPL:

club=df.groupby("Club")["Name"].count().reset_index().sort_values(by="Name",ascending=False).head(10)

px.bar(club,y="Club",x="Name",color="Club",title="Top 10 clubs with most players in EPL",template="plotly_dark",orientation="h",
       labels={"Name":"Count of Players","Club":"Club Name"},text="Name")

In [19]:
#10. players age distribution:

px.histogram(df,x="Age",title="Players Age Distribution",template="plotly_dark",labels={"Age":"Age of Players","count":"Count of Players"})

In [20]:
#11. club's players age means:

p_mean=df.groupby("Club")["Age"].mean().reset_index().sort_values(by="Age",ascending=False).head(10)


fig=px.bar(p_mean,y="Club",x="Age",color="Club",title="Top 10 clubs with highest average age of players",template="plotly_dark",orientation="h",
       labels={"Age":"Average Age of Players","Club":"Club Name"},color_discrete_sequence=px.colors.qualitative.Pastel,width=800,height=500,text="Age",hover_data=["Age"])
fig.update_layout(
            width=1010,
            height=600,
            font_family="Lato",
            title_font_family="Lato",
            title_font_color="white")
fig.update_xaxes(title='Date', tickmode='linear',
                tickfont=dict(family='Lato', size=12))


In [21]:
#12. which players have most yellow and red cards:

df["Total_Cards"]=df["Yellow_Cards"]+df["Red_Cards"]

cards=df.groupby("Name")["Total_Cards"].sum().reset_index().sort_values(by="Total_Cards",ascending=False).head(10)

px.bar(cards,x="Name",y="Total_Cards",color="Name",title="Top 10 players with most yellow and red cards",template="plotly_dark",
       labels={"Total_Cards":"Total Cards","Name":"Player Name"},text="Total_Cards",width=800,height=500)

In [22]:
#Outlier Detection:

vars=["Passes_Attempted","Pass Accuracy","MinPerMacth","MinPerGoal","Yellow_Cards"]

for i in df[vars]:
    px.box(df,y=i,title="Boxplot for "+i,template="plotly_dark",width=600,height=600).show()

In [24]:
#Preperation of data for modelling:

df=df.drop("MinPerGoal",axis=1) #min per goal column is dropped because it has ifinite values.
cat=df.select_dtypes(include="object")
X=df.drop(cat,axis=1)
y=df["Goals"]



#casting float to int:
float_cols = df.select_dtypes(include=['float64']).columns
df[float_cols]=df[float_cols].astype(int)

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)


#models:
models = [('LR', LinearRegression()),
          ('KNN', KNeighborsRegressor()),
          ('CART', DecisionTreeRegressor()),
          ('RF', RandomForestRegressor()),
          ('GBM', GradientBoostingRegressor()),
          ("LOG",LogisticRegression()),
          ("LGBM",LGBMRegressor()),
          ("XGBoost", XGBRegressor())]

# evaluate each model in turn
for name, regressor in models:
    rmse = np.mean(np.sqrt(-cross_val_score(regressor, X, y, cv=5, scoring="neg_mean_squared_error")))
    print(f"RMSE: {round(rmse, 4)} ({name}) ")

# I will use LGBMRegressor for modelling because it has the lowest RMSE value.

RMSE: 0.0 (LR) 
RMSE: 2.8845 (KNN) 
RMSE: 0.2001 (CART) 
RMSE: 0.207 (RF) 
RMSE: 0.1203 (GBM) 
RMSE: 3.3541 (LOG) 
RMSE: 0.6582 (LGBM) 
RMSE: 0.0889 (XGBoost) 


In [ ]:
#building model:
lgbm=LGBMRegressor().fit(X_train,y_train)

#prediction:
y_pred=lgbm.predict(X_test).round()

In [ ]:
#evaluation:

# calculate accuracy
print("Accuracy:\n"+str(accuracy_score(y_test,y_pred)))

# confusion matrix
print("Confusion Matrix:\n"+str(confusion_matrix(y_test,y_pred)))

# Rmse score:
print("Rmse:\n"+str(np.sqrt(mean_squared_error(y_test,y_pred))))

# R2 score:
print("R2 Score:\n"+str(r2_score(y_test,y_pred)))

# mean_absolute_error:
print("mean_absolute_error:\n"+str(mean_absolute_error(y_test,y_pred)))

# Classification report 
print(classification_report(y_test,y_pred))


Accuracy:
0.95
Confusion Matrix:
[[84  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 37  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 14  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  5  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  4  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  4  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  2  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0]]
Rmse:
0.4031128874149275
R2 Score:
0.9773000109134563
mean_absolute_error:
0.075
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        84
         1.0       1.00      1.00      1.00        37
  

In [ ]:
#Model Tuning:

lgbm_params={"learning_rate":[0.001,0.01,0.1,0.05],
             "max_depth":[3,5,8]}

lgbm_cv=GridSearchCV(lgbm,lgbm_params,cv=10,n_jobs=-1,verbose=2)
lgbm_cv.fit(X_train,y_train)

print("En iyi parametreler:"+str(lgbm_cv.best_params_))

Fitting 10 folds for each of 12 candidates, totalling 120 fits
En iyi parametreler:{'learning_rate': 0.1, 'max_depth': 8}


In [ ]:
#Final Model:

lgbm_tuned=LGBMRegressor(learning_rate=0.1,max_depth=8)
lgbm_tuned.fit(X_train,y_train)

y_pred_t=lgbm_tuned.predict(X_test).round(0)

# calculate accuracy
print("Accuracy:\n"+str(accuracy_score(y_test,y_pred_t)))

# confusion matrix
print("Confusion Matrix:\n"+str(confusion_matrix(y_test,y_pred_t)))

# Rmse score:
print("Rmse:\n"+str(np.sqrt(mean_squared_error(y_test,y_pred_t))))

# R2 score:
print("R2 Score:\n"+str(r2_score(y_test,y_pred_t)))

# mean_absolute_error:
print("mean_absolute_error:\n"+str(mean_absolute_error(y_test,y_pred_t)))

# Classification report 
print(classification_report(y_test,y_pred_t))


Accuracy:
0.95
Confusion Matrix:
[[84  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 37  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 14  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  5  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  4  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  4  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  2  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0]]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        84
         1.0       1.00      1.00      1.00        37
         2.0       1.00      1.00      1.00        14
         3.0       1.00      

In [ ]:
#Comparison of Actual and Predicted Values:
df2=pd.DataFrame({"Actual":y_test,"Predicted":y_pred_t})
df2.head(25)

,Actual,Predicted
6,0,0.0
488,1,1.0
104,1,1.0
486,0,0.0
448,0,0.0
222,1,1.0
237,16,12.0
433,0,0.0
347,1,1.0
72,0,-0.0


In [ ]:
#visualization of actual and predicted values:

px.scatter(df2,x="Actual",y="Predicted",title="Actual vs Predicted Values",labels={"Actual":"Actual Values","Predicted":"Predicted Values"},
           width=800,height=500,size="Actual",color="Actual",template="plotly_dark",hover_data=["Actual","Predicted"])

In [ ]:
sns.displot(x=y_test,y=y_pred_t,kind="kde",fill=True)
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title("Actual vs Predicted")